In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

#----------------------------------------------------------
# ハイパーパラメータなどの設定値
num_epochs = 10         # 学習を繰り返す回数
num_batch = 100         # 一度に処理する画像の枚数
learning_rate = 0.001   # 学習率
image_size = 28*28      # 画像の画素数(幅x高さ)

# GPU(CUDA)が使えるかどうか？
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#----------------------------------------------------------
# 学習用／評価用のデータセットの作成

# 変換方法の指定
transform = transforms.Compose([
    transforms.ToTensor()
    ])

# MNISTデータの取得
# https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST
# 学習用
train_dataset = datasets.MNIST(
    './data',               # データの保存先
    train = True,           # 学習用データを取得する
    download = True,        # データが無い時にダウンロードする
    transform = transform   # テンソルへの変換など
    )
# 評価用
test_dataset = datasets.MNIST(
    './data', 
    train = False,
    transform = transform
    )

# データローダー
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = num_batch,
    shuffle = True)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,     
    batch_size = num_batch,
    shuffle = True)

#----------------------------------------------------------
# ニューラルネットワークモデルの定義
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()

        # 各クラスのインスタンス（入出力サイズなどの設定）
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, output_size)

    def forward(self, x):
        # 順伝播の設定（インスタンスしたクラスの特殊メソッド(__call__)を実行）
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#----------------------------------------------------------
# ニューラルネットワークの生成
model = Net(image_size, 10).to(device)

#----------------------------------------------------------
# 損失関数の設定
criterion = nn.CrossEntropyLoss() 

#----------------------------------------------------------
# 最適化手法の設定
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 

#----------------------------------------------------------
# 学習
model.train()  # モデルを訓練モードにする

for epoch in range(num_epochs): # 学習を繰り返し行う
    loss_sum = 0

    for inputs, labels in train_dataloader:

        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizerを初期化
        optimizer.zero_grad()

        # ニューラルネットワークの処理を行う
        inputs = inputs.view(-1, image_size) # 画像データ部分を一次元へ並び変える
        outputs = model(inputs)

        # 損失(出力とラベルとの誤差)の計算
        loss = criterion(outputs, labels)
        loss_sum += loss

        # 勾配の計算
        loss.backward()

        # 重みの更新
        optimizer.step()

    # 学習状況の表示
    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss_sum.item() / len(train_dataloader)}")

    # モデルの重みの保存
    torch.save(model.state_dict(), 'model_weights.pth')

#----------------------------------------------------------
# 評価
model.eval()  # モデルを評価モードにする

loss_sum = 0
correct = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:

        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)

        # ニューラルネットワークの処理を行う
        inputs = inputs.view(-1, image_size) # 画像データ部分を一次元へ並び変える
        outputs = model(inputs)

        # 損失(出力とラベルとの誤差)の計算
        loss_sum += criterion(outputs, labels)

        # 正解の値を取得
        pred = outputs.argmax(1)
        # 正解数をカウント
        correct += pred.eq(labels.view_as(pred)).sum().item()

print(f"Loss: {loss_sum.item() / len(test_dataloader)}, Accuracy: {100*correct/len(test_dataset)}% ({correct}/{len(test_dataset)})")

Epoch: 1/10, Loss: 0.6682661437988281
Epoch: 2/10, Loss: 0.2675306447347005
Epoch: 3/10, Loss: 0.21094985961914062
Epoch: 4/10, Loss: 0.17717769622802734
Epoch: 5/10, Loss: 0.15273826599121093
Epoch: 6/10, Loss: 0.13321675618489584
Epoch: 7/10, Loss: 0.11749045054117839
Epoch: 8/10, Loss: 0.10489582061767579
Epoch: 9/10, Loss: 0.09360068639119466
Epoch: 10/10, Loss: 0.08430411020914713
Loss: 0.10421312332153321, Accuracy: 96.79% (9679/10000)


In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# ベクトルのサイズ
N = 10

# NumPyでホスト側データを用意
a = np.random.randn(N).astype(np.float32)
b = np.random.randn(N).astype(np.float32)

# デバイス上のメモリを確保して転送
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(a.nbytes)

cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# CUDAカーネルの定義
mod = SourceModule("""
__global__ void add_vec(float *a, float *b, float *c, int N) {
    int idx = threadIdx.x;
    if (idx < N) {
        c[idx] = a[idx] + b[idx];
    }
}
""")

# カーネル関数を取得
add_vec = mod.get_function("add_vec")

# カーネル実行
add_vec(a_gpu, b_gpu, c_gpu, np.int32(N), block=(N,1,1), grid=(1,1))

# 結果をホスト側へ転送
c = np.empty_like(a)
cuda.memcpy_dtoh(c, c_gpu)

# 結果の表示
print("a =", a)
print("b =", b)
print("c =", c)
print("CPUでの計算結果と一致:", np.allclose(c, a + b))


a = [-1.1275737  -0.4672829  -0.16977865 -0.8891911  -0.13462514  0.5725335
  0.6302399   1.5283407  -0.52328986 -1.34725   ]
b = [ 0.20108047 -0.46554175  1.1118873  -0.2851398   3.1001132   0.8838516
  0.94663423  1.2293041   0.16702989  0.60649157]
c = [-0.9264933  -0.9328246   0.9421087  -1.174331    2.965488    1.4563851
  1.5768741   2.7576447  -0.35625997 -0.7407584 ]
CPUでの計算結果と一致: True
